In [ ]:
!pip install wandb

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback

In [6]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
wandb.init(project="Mnist", entity="gharabadiyan")

wandb: Currently logged in as: gharabadiyan. Use `wandb login --relogin` to force relogin


In [8]:
config = wandb.config
config.learning_rate = 0.001

## Mnist Dataset

In [9]:
mnist = tf.keras.datasets.mnist

(X_train, Y_train),(X_test, Y_test) = mnist.load_data()

X_train, X_test = X_train / 255.0, X_test / 255.0


11501568/11490434 [==============================] - 0s 0us/step


## MLP

In [10]:

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=5,callbacks=[WandbCallback()])


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2955 - accuracy: 0.9150 - _timestamp: 1654666143.0000 - _runtime: 111.0000
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1435 - accuracy: 0.9583 - _timestamp: 1654666148.0000 - _runtime: 116.0000
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1087 - accuracy: 0.9672 - _timestamp: 1654666154.0000 - _runtime: 122.0000
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0885 - accuracy: 0.9733 - _timestamp: 1654666159.0000 - _runtime: 127.0000
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0764 - accuracy: 0.9764 - _timestamp: 1654666164.0000 - _runtime: 132.0000


In [11]:
model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0736 - accuracy: 0.9773


[0.07357104867696762, 0.9772999882698059]

In [12]:
model.save('MLP_Mnist.h5')

## MLP+CNN

In [13]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state= 8)

model = tf.keras.models.Sequential([
                                    
        #استخراج ویژگی                           
        layers.Conv2D(32,(3,3),activation='relu',input_shape=(28, 28,1)),
        layers.MaxPool2D(),
        layers.Conv2D(64,(3,3),activation='relu'),                            
        layers.MaxPool2D(),
        layers.Conv2D(64,(5,5),activation='relu'),

        layers.Flatten(),

        #همون شبکه قدیمی
        layers.Dense(48,activation='relu'),
        layers.Dense(10,activation='softmax')
])
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=config.learning_rate),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [14]:
model.fit(X_train,Y_train,epochs=28,validation_data=(X_val,Y_val),callbacks=[WandbCallback()])

Epoch 1/28
1500/1500 [==============================] - 49s 33ms/step - loss: 2.2245 - accuracy: 0.2633 - val_loss: 2.0758 - val_accuracy: 0.4602 - _timestamp: 1654666276.0000 - _runtime: 244.0000
Epoch 2/28
1500/1500 [==============================] - 49s 33ms/step - loss: 1.4677 - accuracy: 0.6781 - val_loss: 0.7864 - val_accuracy: 0.8066 - _timestamp: 1654666325.0000 - _runtime: 293.0000
Epoch 3/28
1500/1500 [==============================] - 48s 32ms/step - loss: 0.5673 - accuracy: 0.8476 - val_loss: 0.4561 - val_accuracy: 0.8689 - _timestamp: 1654666373.0000 - _runtime: 341.0000
Epoch 4/28
1500/1500 [==============================] - 50s 33ms/step - loss: 0.3884 - accuracy: 0.8870 - val_loss: 0.3517 - val_accuracy: 0.8992 - _timestamp: 1654666423.0000 - _runtime: 391.0000
Epoch 5/28
1500/1500 [==============================] - 50s 33ms/step - loss: 0.3207 - accuracy: 0.9047 - val_loss: 0.3054 - val_accuracy: 0.9120 - _timestamp: 1654666473.0000 - _runtime: 441.0000
Epoch 6/28
1500

In [15]:
model.evaluate(X_test,Y_test)

313/313 [==============================] - 4s 11ms/step - loss: 0.0805 - accuracy: 0.9759


[0.08054729551076889, 0.9758999943733215]

In [16]:
model.save('MLP_CNN_Mnist.h5')